# Statistical Techniques for Classifiers Comparison with Python

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import random as rn
import os
from sklearn import model_selection
import tensorflow as tf

In [ ]:
# Seed for reproducability
seed = 1234
np.random.seed(seed)
rn.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
# Load dataset
mnist = tf.keras.datasets.mnist
train_data, test_data = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
X = np.concatenate((train_data[0], test_data[0]))
Y = np.concatenate((train_data[1], test_data[1]))

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(
X, Y, test_size=0.15, random_state=42, stratify = Y)

In [ ]:
# Train val split
train_images, val_images, train_labels, val_labels = train_test_split(
train_images, train_labels, test_size=0.15, random_state=50, stratify = train_labels)

In [ ]:
# Reshape the labels and encode them categorically.
y_train = tf.keras.utils.to_categorical(train_labels)
y_test = tf.keras.utils.to_categorical(test_labels)
y_val =  tf.keras.utils.to_categorical(val_labels)

In [ ]:
# Reshape and normalize the images.
X_train = train_images.reshape((train_images.shape[0], 784))
X_train = X_train.astype('float32') / 255
X_test = test_images.reshape((test_images.shape[0], 784))
X_test = X_test.astype('float32') / 255
X_val = val_images.reshape((val_images.shape[0], 784))
X_val = X_val.astype('float32') / 255


## Testing on Data

In [ ]:
data = pd.read_csv('sudoku_data_updated.csv')

In [ ]:
# be careful with this because if you run it multiple times, it will destroy the data
data['difficulty'] = data['difficulty'] / 2
data['difficulty'] = data['difficulty'].astype(int)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
data_1 = pd.concat([data.iloc[:1800],data.iloc[5400:5600]], ignore_index=True) #difficulty low
data_2 = pd.concat([data.iloc[1800:3600],data.iloc[5600:5800]]) #difficulty medium
data_3 = pd.concat([data.iloc[3600:5400],data.iloc[5800:6000]]) # difficulty high


In [ ]:
## Using all
X = data
y = data['difficulty']
X_1 = data_1
y_1 = data_1['difficulty']
X_2 = data_2
y_2 = data_2['difficulty']
X_3 = data_3
y_3 = data_3['difficulty']

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=51)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=52)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, test_size=0.2, random_state=53)

X_train, X_test, y_train, y_test = pd.concat([X_train_1,X_train_2,X_train_3], ignore_index=True),pd.concat([X_test_1,X_test_2,X_test_3],ignore_index=True), pd.concat([y_train_1,y_train_2,y_train_3],ignore_index=True),pd.concat([y_test_1,y_test_2,y_test_3],ignore_index=True)

In [ ]:
def predict_rf(features, X_train, y_train, X_test, y_test):
  classifier = RandomForestClassifier(n_estimators=160, random_state=50)
  classifier.fit(X_train[features], y_train)
  y_pred = classifier.predict(X_test[features])
  return y_pred

In [ ]:
# no solving strategies
features = ['NoC','Poss','stddev','Var','r0','r1','r2','r3','Clusters']
y_pred_1 = predict_rf(features, X_train, y_train, X_test, y_test)

In [ ]:
# all solving strategies
features = ['NoC','Poss','stddev','Var','r0','r1','r2','r3','Clusters',
            'NoC after pass', 'NoC after 2pass','NoC diff','NoC after passes',
            'Pairs','Pair Red','Pair Red2','Pair Diff2','Pairs Poss']
y_pred_3 = predict_rf(features, X_train, y_train, X_test, y_test)

In [ ]:
# testing one solving strat at a time
variable = 'Pairs Poss'
features = ['NoC','Poss','stddev','Var','r0','r1','r2','r3','Clusters',variable]
y_pred_2 = predict_rf(features, X_train, y_train, X_test, y_test)

In [ ]:
ct = pd.crosstab(y_pred_1, y_pred_2, margins=True)
print(ct)

col_0    0    1    2   All
row_0                     
0      256   77   77   410
1       84  201  110   395
2       66   93  236   395
All    406  371  423  1200


In [ ]:
ctab = ct.iloc[:-1,:-1]
contingency_table = ctab.copy()
print(contingency_table)

col_0    0    1    2
row_0               
0      256   77   77
1       84  201  110
2       66   93  236


In [ ]:
# @title Code for running the tests
from scipy.stats import norm
from scipy import stats


def apply_sweep(x, MARGIN, FUN=np.subtract, STATS=None, check_margin=True, **kwargs):
    '''
    A Python version of the R function apply_sweep() from the basic R syntax.
    The sweep R function applies an operation (e.g. + or -) to a data matrix by row or by column.


    Parameters:
    -----------------------
    x: Typically a matrix.
    MARGIN: Specifies typically whether the operation should be applied by row or by column. MARGIN = 1 operates by row; MARGIN = 2 operates by column.
    STATS: Specifies usually the value that should be used for the operation (e.g. the value that should be added or subtracted).
    FUN: The operation that should be carried out (e.g. + or -).

    Returns:
    -----------------------
    A matrix with the same dimensions as x with the operation applied by row or by column.
    '''

    if STATS is None:
        STATS = np.array([], dtype=x.dtype)

    dims = np.shape(x)

    if isinstance(MARGIN, str):
        dn = np.array(x).dtype.names

        if dn is None:
            raise ValueError("'x' must have named dimnames")

        MARGIN = np.where(dn == MARGIN)[0][0]

        if np.any(np.isnan(MARGIN)):
            raise ValueError("not all elements of 'MARGIN' are names of dimensions")

    if check_margin:
        dimmargin = dims[MARGIN]
        dimstats = pd.DataFrame(STATS).shape
        lstats = np.size(STATS)

        if lstats > np.prod(dimmargin):
            print("Warning: STATS is longer than the extent of 'dim(x)[MARGIN]'")
        elif len(dimstats) == 0:
            cumDim = np.append(1, np.cumprod(dimmargin))
            upper = np.min(cumDim[cumDim >= lstats])
            lower = np.max(cumDim[cumDim <= lstats])

            if lstats and (upper % lstats != 0 or lstats % lower != 0):
                print("Warning: STATS does not recycle exactly across MARGIN")
        else:
            dimmargin = np.array(dimmargin).reshape(-1)
            dimstats = np.array(dimstats)[:-1]

            if len(dimstats) != len(dimmargin) or not np.any(dimstats == dimmargin):
                print("Warning: length(STATS) or dim(STATS) do not match dim(x)[MARGIN]")

    perm = np.concatenate((np.arange(MARGIN), np.arange(MARGIN+1, len(dims)), [MARGIN]))
    return FUN(x, STATS).transpose(perm).T




# %%

# KOHEN'S KAPPA

def kappa(x, weights=['Equal-Spacing', 'Fleiss-Cohen']):
    '''
    Computes two agreement rates: Cohen's kappa and weighted kappa, and confidence bands.
    A Python version of the R function Kappa() from the vcd package.

    Parameters:
    -----------------------
    x: array_like or pandas.DataFrame
        If `x` is a matrix, it should be a KxK matrix of counts of
        assignments of objects to K categories.
    weights:veither one of the character strings given in the default value, or a user-specified matrix with same dimensions as x.


    Returns:
    -----------------------
    A dictionary containing the following:
    - 'Unweighted': numeric vector of length 2 with the kappa statistic (value component), along with Approximate Standard Error (ASE component).
    - 'Weighted': idem for the weighted kappa.
    - 'CI': matrix with lower and upper bounds of the 95% confidence interval for the unweighted and weighted kappa.

    '''
    if isinstance(weights, str):
        weights = [weights]
    d = np.diag(x)
    n = np.sum(x).sum()
    nc = x.shape[1]
    colFreqs = np.sum(x, axis=0)/n
    rowFreqs = np.sum(x, axis=1)/n

    def kappa_val(po, pc):
        return (po - pc)/(1 - pc)
    def std(p, pc, kw, W=None):
        dot1 = np.dot(W, np.sum(p, axis=0))
        dot2 = np.dot(W, np.sum(p, axis=1) * (1 - kw))
        inner1 = np.inner(1, 1-pc)
        inner2 = np.inner(1-pc, 1)
        sweep1 = apply_sweep( W, MARGIN = 1, STATS = dot1 * (1 - kw))
        return  np.sqrt(((np.sum(p * apply_sweep(sweep1, MARGIN=0, STATS = dot2)**2)).sum()- (kw - pc * (1 - kw)) **2) /(inner1 * inner2)/n)

    # unweighted kappa
    po = np.sum(d)/n
    pc = np.inner(colFreqs, rowFreqs)
    k = kappa_val(po, pc)
    W0 = np.diag(np.ones(nc))
    s = std(x/n, pc, k, W0)

    # weighted kappa
    if isinstance(weights, list):
        if weights[0] == 'Equal-Spacing':
            W = 1 - np.abs(np.subtract.outer(np.arange(1, nc+1), np.arange(1, nc+1)))/(nc-1)
        else:
            W = 1 - np.power(np.abs(np.subtract.outer(np.arange(1, nc+1), np.arange(1, nc+1)))/(nc-1), 2)
    else:
        W = weights

    pow = np.sum(W * x).sum()/n
    pcw = np.sum(W * np.outer(colFreqs, rowFreqs))
    kw = kappa_val(pow, pcw)
    sw = std(x/n, pcw, kw, W)

    #compute z-score and p-value
    # tab <- rbind(x$Unweighted, x$Weighted)
    tab = np.vstack((np.array([k, s]), np.array([kw, sw])))
    z = tab[:,0] / tab[:,1]
    # tab <- cbind(tab, z, `Pr(>|z|)` = 2 * pnorm(-abs(z)))
    tab = np.c_[tab, z, 2 * norm.cdf(-abs(z))]

    CI = True
    if CI:
        level = 0.95
        q = norm.ppf((1 + level)/2)
        lower = tab[:,1] - q * tab[:,2]
        upper = tab[:,1] + q * tab[:,2]
        tab_ci = np.c_[tab, lower, upper]


    return {'Unweighted': {'kappa': k, 'std': s, 'z': z[0], 'p': tab[0,3]},
            'Weighted': {'kappa': kw, 'std': sw, 'z': z[1], 'p': tab[1,3]},
            'CI': {'lower': lower, 'upper': upper}}


# STUART-MAXWELL TEST

def StuartMaxwellTest(x, y=None, BhapkarTest=False):
    """
    Stuart-Maxwell test for marginal homogeneity of a KxK matrix of assignments
    of objects to K categories or an nx2 or 2xn matrix of category scores for n
    data objects by two raters. The statistic is distributed as Chi-square with
    K-1 degrees of freedom.

    Parameters:
    -----------
    x : array_like or pandas.DataFrame
        If `x` is a matrix, it should be a KxK matrix of counts of
        assignments of objects to K categories.
    y : array_like, optional
        If `x` is a data frame, `y` should not be provided. Otherwise,
        `y` should be a KxK matrix with the same counts as `x`.

    Returns:
    --------
    result : dict
        A dictionary containing the following:
        - 'method': str, the name of the test
        - 'data.name': str, the name of the data
        - 'statistic': float, the test statistic
        - 'parameter': int, the degrees of freedom
        - 'p.value': float, the p-value of the test
        - 'N': int, the total number of observations
    """

    if isinstance(x, np.ndarray or pd.DataFrame):
        r = x.shape[0]
        if r < 2 or x.shape[1] != r:
            raise ValueError("'x' must be square with at least two rows and columns")
        if (x < 0).any().any() or np.isnan(x).any().any():
            raise ValueError("all entries of 'x' must be nonnegative and finite")
        DNAME = "x"
    else:
        if y is None:
            raise ValueError("if 'x' is not a matrix, 'y' must be given")
        if y is not None and not isinstance(y, np.ndarray or pd.DataFrame):
            raise TypeError("'y' must be a matrix")
        if len(x) != len(y):
            raise ValueError("'x' and 'y' must have the same length")
        DNAME = " and ".join([str(x), str(y)])
        OK = np.logical_and(~np.isnan(x), ~np.isnan(y))
        x = np.asarray(x, dtype=np.int64)[OK]
        y = np.asarray(y, dtype=np.int64)[OK]
        r = np.unique(x).shape[0]
        if r < 2 or np.unique(y).shape[0] != r:
            raise ValueError("'x' and 'y' must have the same number of levels (minimum 2)")
        x = np.asarray(np.histogram2d(x, y, bins=r)[0], dtype=np.int64)

    # get the marginals
    rowsums = x.sum(axis=1)
    colsums = x.sum(axis=0)
    #  If you have perfect agreement then you want something along the lines of:
    equalsums_rows = np.diag(x) == rowsums
    equalsums_cols = np.diag(x) == colsums

    equalsums = np.logical_and(equalsums_rows, equalsums_cols)

    if equalsums.any():
        # dump any categories with perfect agreement
        x = x[~equalsums][:, ~equalsums]
        # bail out if too many categories have disappeared
        if x.shape[0] < 2:
            raise ValueError("Too many equal marginals, cannot compute")
        # get new marginals
        rowsums = x.sum(axis=1)
        colsums = x.sum(axis=0)

    # use K-1 marginals
    Kminus1 = rowsums.shape[0] - 1
    smd = rowsums[:-1] - colsums[:-1]
    smS = np.zeros((Kminus1, Kminus1))
    if isinstance(x, np.ndarray):
        x = pd.DataFrame(x)
    for i in range(Kminus1):
        for j in range(Kminus1):
            if i == j:
                smS[i,j] = rowsums[i] + colsums[j] - 2 * x.iloc[i,i]
            else:
                smS[i,j] = -(x.iloc[i,j] + x.iloc[j,i])

    N = rowsums.sum()

    if BhapkarTest == False:
        STATISTIC = smd @ np.linalg.inv(smS) @ smd
        PARAMETER = r - 1
        METHOD = "Stuart-Maxwell test"
        # PVAL = stats.chi2.sf(STATISTIC, PARAMETER)
        PVAL = stats.chi2.sf(STATISTIC, PARAMETER)

    elif BhapkarTest == True:
        STATISTIC_pre = smd @ np.linalg.inv(smS) @ smd
        STATISTIC = STATISTIC_pre / ( 1- STATISTIC_pre / N)
        PARAMETER = r - 1
        METHOD = "Bhapkar test"
        PVAL = stats.chi2.sf(STATISTIC, PARAMETER)


    RVAL = {"statistic": STATISTIC, "parameter/dof": PARAMETER,
            "p.value": PVAL, "method": METHOD, "data.name": DNAME,
            "N": N}
    return RVAL


In [ ]:
# Compute Kohen's K
KAPPA = kappa(contingency_table)
print(KAPPA['Unweighted'])
print(KAPPA['Weighted'])

{'kappa': 0.36619058036690944, 'std': 0.021343131937481063, 'z': 17.157302941272434, 'p': 5.542857116529104e-66}
{'kappa': 0.39708280835735976, 'std': 0.022509121238915804, 'z': 17.640973369979772, 'p': 1.193866502429738e-69}


In [ ]:
# Stuart-Maxwell test
contingency_table = contingency_table.to_numpy()
SM = StuartMaxwellTest(contingency_table, BhapkarTest=False)
print(SM)
# Bhapkar test
BP = StuartMaxwellTest(contingency_table, BhapkarTest=True)
print(BP)

{'statistic': 2.5000295037469757, 'parameter/dof': 2, 'p.value': 0.28650057040884724, 'method': 'Stuart-Maxwell test', 'data.name': 'x', 'N': 1200}
{'statistic': 2.5052488337458025, 'parameter/dof': 2, 'p.value': 0.28575387463462465, 'method': 'Bhapkar test', 'data.name': 'x', 'N': 1200}
